# Generate actors

Generate fake user profiles including their profile pictures. 

Specify the number of profiles you would like to generate in the variable `num_profiles`. For example, `num_profiles = 100`. If you would like locations to be limited to the USA, then set `limit_to_USA` to `True`; otherwise, setting it to `False` will allow for locations to be anywhere in the world. If you would like to change the image size, you may do so with the variable `image_size`.

The bios are filled with random occupations (from https://github.com/johnlsheridan/occupations/blob/master/occupations.csv but saved locally in the `internal/` folder). Everything else is generated using the Random User API.

## Run the program

Run the program by selecting *Kernel > Restart Kernel and Run All Cells...* in the menu bar at the top left of the screen. Results will be in `output > csv_files > actors.csv` and `output > profile_pictures`. You will see the words, "Profiles generated!!" at the bottom of the notebook when the code is finished running.

In [ ]:
num_profiles = 10  # the number of profiles to generate
limit_to_USA = False  # options: True or False
image_size = 'large'  # options: 'thumbnail', 'medium', or 'large'

In [ ]:
import numpy as np
import pandas as pd
import requests
import random
from random_address import real_random_address

In [ ]:
response = requests.get('https://randomuser.me/api/?results=' + str(num_profiles))
results = response.json()['results']

In [ ]:
profile_data = {'username': [],
                'name': [],
                'age': [],
                'picture': [],
                'gender': [],
                'bio': [],
                'location': [],
                'class': []
}

# get list of occupations
with open('internal/occupations.txt') as f:
    lines = f.read()
occupations = lines.split('\n')

for i in range(len(results)):
    # download images
    image_data = requests.get(results[i]['picture'][str(image_size)]).content
    image_name = 'output/profile_pictures/profile_img_' + str(i) + '.png'
    with open(image_name, 'wb') as handler:
        handler.write(image_data)

    profile_data['username'].append(results[i]['login']['username'])
    profile_data['name'].append(results[i]['name']['first'] + " " + results[i]['name']['last'])
    profile_data['age'].append(results[i]['dob']['age'])
    profile_data['picture'].append('profile_img_' + str(i) + '.png')
    profile_data['gender'].append(results[i]['gender'])
    profile_data['bio'].append(random.choice(occupations))
    if limit_to_USA:
        while True:
            this_address = real_random_address()
            if 'city' in this_address.keys() and 'state' in this_address.keys():
                break
        profile_data['location'].append(this_address['city'] + ", " + this_address['state'])
    else:
        profile_data['location'].append(results[i]['location']['city'] + ", " + results[i]['location']['country'])
    profile_data['class'].append('normal')

profiles_df = pd.DataFrame(profile_data)
profiles_df.to_csv('output/csv_files/actors.csv', index=False)

print("\n\n\nProfiles generated!!")